In [1]:
!pip install open_clip_torch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00


In [26]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import open_clip
import glob
from collections import defaultdict

# --- Configuration ---
base_dir = '/kaggle/input/keyframe-real/keyframes'   # Each folder = 1 video
output_dir = '/kaggle/working/features'    # Save 1 .npy per image
os.makedirs(output_dir, exist_ok=True)

# --- Load OpenCLIP ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name='ViT-H-14-quickgelu',
    pretrained='dfn5b',
    device=device
)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-31): 32 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=Tru

In [24]:
videos = glob.glob(os.path.join(base_dir,'*','*','*.jpg'))
if not videos:
    print("[ERROR] No videos found.")

print(f"[INFO] Found {len(videos)} videos to process.")
videos[:10]

[INFO] Found 17585 videos to process.


['/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_2449.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_5149.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_124.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_2388.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_5033.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_5749.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_7055.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_6209.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_2773.jpg',
 '/kaggle/input/keyframe-real/keyframes/L26/L26_V215/L26_V215_5269.jpg']

In [ ]:
batch_size=256

folder_to_images = defaultdict(list)
for img_path in videos:
    folder_to_images[os.path.dirname(img_path)].append(img_path)

for folder_path, image_files in tqdm(folder_to_images.items(), desc="Processing folders"):
    # Create matching output subfolder
    rel_path = os.path.relpath(folder_path, base_dir)
    save_folder = os.path.join(output_dir, rel_path)
    os.makedirs(save_folder, exist_ok=True)

    # Sort image paths for consistent processing
    image_files = sorted(image_files)

    # Process in batches
    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        batch_images = []
        valid_files = []

        for img_path in batch_files:
            try:
                image = Image.open(img_path).convert('RGB')
                image_tensor = preprocess(image)
                batch_images.append(image_tensor)
                valid_files.append(img_path)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

        if not batch_images:
            continue

        # Encode features
        input_tensor = torch.stack(batch_images).to(device)
        with torch.no_grad():
            features = model.encode_image(input_tensor)
            features = features / features.norm(dim=-1, keepdim=True)

        # Save each feature with same relative path
        for img_path, feat in zip(valid_files, features):
            rel_img_path = os.path.relpath(img_path, base_dir)
            save_path = os.path.join(output_dir, os.path.splitext(rel_img_path)[0] + '.npy')
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            np.save(save_path, feat.cpu().numpy())

Processing folders:   8%|▊         | 14/175 [01:37<17:33,  6.55s/it]